In [ ]:
%run -i ../python/common.py

# cleanup any old state
bashCmds('''[[ -d mydir ]] && rm -rf mydir
[[ -a myfile ]] && rm myfile
[[ -a errors ]] && rm errors 
[[ -a mydate ]] && rm mydate
[[ -a mynewdir ]] && rm -rf mynewdir
[[ -a anotherfile ]] && rm anotherfile
[[ -a mybin ]] && rm -rf  mybin''')

closeAllOpenTtySessions()
bash = BashSession()

generated="~/myfile ~/errors ~/mydate ~/mydir ~/mynewdir ~/out"

# remember to avoid something having a history entry put a space at the beginning of the line
#bash.run(" history")

# The Shell - Part I: Having an ASCII conversation with the OS


As we read in the Unix [introduction](../unix/intro.ipynb) a key feature of Unix is its command line interface that was developed for [ASCII terminals](../unix/terminal.ipynb) devices. Our goal in this chapter is to understand the general model of the shell and how to start working in the terminal environment.  If you have not done so be sure to have read the sections on the Unix [Kernel](../unix/intro.ipynb#UnixKernel_sec), [User Programs](../unix/intro.ipynb#UnixUser_sec), how we [visualize a running Unix system](../unix/intro.ipynb#UnixViz_sec), the introduction to [Terminal Emulators](../unix/terminal.ipynb#TerminalEmulators_sec) and the chapter introducing [Files and Directories](../unix/files.ipynb).

> <img style="display: inline; margin: 1em 1em 0px 0px;" align="left" width="40" src="../images/fyi.svg"> <p style="background-color:powderblue;"> It is a good idea to read this chapter with a terminal connection open so that you can explore the material as you read it.  Instructions and guidance on how to do this can be found in [Terminal Emulators](../unix/terminal.ipynb#TerminalEmulators_sec).

## Our interface to the Kernel

As discussed the core functionality of the Unix operating system is implemented by the Kernel.  But the kernel is really only responsible for making it easier to run and write application/user programs. The Kernel does not have any support for humans to directly interact with it.  That is where the shell comes in. The shell acts as our primary point of contact/interface not only to the kernel but also all the other programs installed.

### System calls

<img  src="../images/syscalls.png">

The UNIX kernel provides a large set of functions that can be called by a running program.  Remember we call a running program a process on UNIX. The kernel functions that a process can request are called system calls.  In the next part of this book we will look at the  details for how a process can make a request to the kernel to execute a system call.  For the moment it is sufficient to know that each system call, which we identify by names, can only be requested by an already running program.  When a process makes a request to execute a system call it can pass arguments to the system call and the kernel on completion can return values back to the process.  Operating system kernels can provide hundreds of system calls.  The white labeled boxes within the kernel oval, in the diagram to the left, illustrates a small number of important UNIX kernel system calls. 

The UNIX kernel through its system calls provide the ability for a process to:
1. create and manage files and directories,
2. create and manage other process, 
3. manage and work with Input and Output devices
4. communicate between processes both on the same computer and across computers

<img style="float: right; margin: 0em 0em 0px 0px;" width="50%" src="../images/shellsystemcalls2.png">
A core job of the shell is to be the first running process a human user of the computer can use to interact with the kernel to get things done.  In other words a shell has been built to let a human make use of the Operating System kernel, which itself has been written to help us make use of the computer hardware.  It does this by: 1) accepting human oriented commands from a user, 2) executing kernel system calls to get the work of the commands done, and 3) sending a response back.  The shell includes the ability to locate other executable programs and launch new processes from them.    

The graphical human computer interfaces that we use on our smart phones, laptops and desktop computers are designed utilize touch or pointing devices like a mouse, also allow us to; find and launch programs, open files and generally do our daily work.  The primary design of these interfaces are for ease of use.  The shell, however, is text based and is really geared towards programmers.  Specifically its syntax and commands are designed to itself form an "extensible" programming language.  Extensible hear means the ability for it to easily added to by the users.  This language allows a human user to not only use the facilities of the kernel but to also automate their daily tasks and efficiently conduct the various forms of data processing programmer often do. 

## Bash
Over the years there have been many variants of the shell program that have been developed not only for Unix but other operating systems as well (see [comparison of shells](https://en.wikipedia.org/wiki/Comparison_of_command_shells).   While there are differences between the various Unix shell programs the basic model of interaction we cover in this chapter is largely the same across all of them.   However, with respect to the exact syntax and details we will be focus on the [Bourne Again Shell (BASH)](https://en.wikipedia.org/wiki/Bash_(Unix_shell)),  as it is the default shell for Linux which is the version of Unix we use through out this book.

### man bash
It is a Unix tradition that programs installed on the system, like Bash, include documentation on how it is used.  To access the manual pages you use the `man` program eg. `man man` would bring up the documentation about the manual itself.  So `man bash` will bring up the manual page for the bash shell.  You can find detailed coverage of the syntax and how bash works in the manual.  After we cover some basics it will be much easier to read and understand the Bash manual page as it assumes that you understand some basic UNIX and generic shell concepts.  

In [ ]:
bash.run("man bash", height='20em')

## Shell Session
Remembering that ASCII terminals were the original devices created for humans to interactively work with computers the shell is designed to managed and communicate with a human at a terminal. Thus the human and the shell are really exchanging bytes that encoded information in ASCII.  For every new terminal connection software directs the kernel to start an instance of a shell program  to interact with a user at the terminal.  We consider this the be the start of a shell "session".  The session ends when either the communication between the shell and the terminal is disconnected or because the user purposefully exits the shell.  


### Terminal Windows and the Shell
Today of course, we rarely use physical terminals any more.  Rather, on our physical client computers (eg. laptops, desktops, tablets, etc) we can start many terminal emulator windows to establish a shell session for each (see below). Each session will stay active until we exit or close the session and kill the associated shell.  Since each window is connected to its own shell they represent independent "conversations" in which we can do different things concurrently.

In [ ]:
display(HTML(htmlFig(
    [[
        {'src':"../images/UnixL01_SHCHT/041SHLLChat.png",
         'cellwidth': '47.5%'},
        {'src':"../images/terminalwins.png",
         'cellwidth': '52%'}
     ]],
    caption="Figure: Today terminal emulators running on our desktop act act as ASCII terminals. Every new window is a new terminal connection."
)))

The important thing is we don't need to do anything special, simply connecting to the UNIX system, from a terminal, starts a shell for us to interact with.  

> <img style="display:inline; margin: 1em 1em 0px 0px;" align="left" width="40" src="../images/do.svg"> <p style="background-color:lightgreen;">  While you are learning how to work with bash it can be really useful to keep a terminal open in which you keep the bash manual page open while you use another terminal shell session to do your exploring.

## Line oriented conversation

<img style="float: left; margin: 0px 10px 0px 0px;" width="30%" src="../images/shellloopcartoon.png">
The way we interact with the shell can be thought of as a structured, interactive, "conversation". The conversation is a series of back and forth exchanges. We, the user, type in a request in the form of a "command line" and in response the shell takes some actions and generate a particular response, as byte values, which are sent back to the terminal.  The terminal translates the response as ASCII characters displaying/printing the appropriate images of the ascii characters to the screen so we can read it.   

### Lines

Given our use of ASCII what constitutes a line, in Unix, is very precise. It is a series of byte values terminated by a single 'new-line/line-feed' value.  Using an ASCII  table we see that the byte value for the new-line character is `0b10000010` in binary or expressed more concisely in hex as `0xA`. The common short hand notation for the newline byte value is `\n`.  Pressing the *return*  or *enter* key, depending on your keyboard, within a terminal window will generate this value.   We will simply refer to the key that generates a '\n' as the *enter* key.  

When `\n` is received by the shell it will begin working on processing the line as a command.  In this way it is line oriented.  If the user presses `\n` with no other preceding characters this is considered a blank or empty line.  

In the other direction, the terminal emulator is configured to take the appropriate action when it receives a `\n`.  That is to say it moves the cursor to the next lower position in the window (scrolling if configured to do so) and moves it to the left edge of the new line (again scrolling the window if configured to).

    
><img style="display:inline; margin: 1em 1em 0px 0px;" align="left" width="60" src="../images/fyi.svg"> <p style="background-color:powderblue;"> There are two basic modes that the UNIX kernel can use when sending bytes sent from the terminal connection to the program running on the connection; character and line. So far in our case, this program is the shell, later we will see that programs that we run from the shell can be allowed to take over this role.   In character mode the terminal data is sent along to the program running as soon as they are received, including the `\n`.  In line (or buffered mode) the kernel stores up the characters sent until it receives a `\n`.   At which point it sends the complete line with the `\n` to the program running. The default mode is typically line mode.  For the most part it does not really matter to us as the shell's behavior is largely the same as it is a line oriented program that is designed to process whole lines at a time.  However, later on this can be a source of confusion when we are writing programs that are not line oriented, rather they assume that they will see characters as soon as keys are pressed by the user.  

    
### The Prompt

<img style="float: right; margin: 0px 0px 0px 10px;" width="50%" src="../images/UnixL01_SHCHT/04SHLLChat.png">
To visually let us know that the shell is ready for us to send it a line it sends a configurable sequence of characters which we call the prompt string or simply the prompt.  When a new connection is established it sends the prompt to the terminal so that we know that it is ready for the session/conversation to begin.  The classic prompt string is often either the dollar symbol, `$` or greater than sign `>` followed by a single space character.  In our illustrations and the default configuration we use sets up the prompt to be the classic dollar sign followed by a space.

><img style="display: inline; margin: 1em 1em 0px 0px;" align="left" width="60" src="../images/fyi.svg"> <p style="background-color:powderblue;"> Today most system default to much more complicated prompt strings.  Where the prompt might state your user name, the date or time, what your current working directory is, etc.   Later when we learn about shell variables we will find out that there is a special variable `PS1` who's value is used to generate the prompt string every time the shell want to send it.  Given that every terminal session/window is connected to separate shells it can be very useful to set each with a unique prompt to help keep you organized eg. `export PS1='Term 1> '`.
If you are interested in the details use the following `man bash` to pull up the manual page for bash and look for the section called "Prompting".    

## Our first shell session

Before we more formally explain the operation of the shell lets poke it and see how it behaves.
Perhaps the simplest thing we can do is send it an empty line by pressing *enter* on its own.
What we should find is that the terminal will move to the next line and display the prompt again. 
But we want to be a little more careful to think about what happened behind the scenes so that we can build a more complete model in our mind about how the shell and terminal interactions work.

In [ ]:
display(HTML(htmlFig(
    [
        [
            {'src':"../images/UnixL01_SHCHT/06SHLLChat.png",
             'caption':'A: Press Enter', 
             'border': '1px solid black',
             'padding':'1px',
             'cellwidth':'33.33%'
            },
            {'src':"../images/UnixL01_SHCHT/07SHLLChat.png",
             'caption':'B: Shell "blank line" input processing' , 
             'border': '1px solid black',
             'padding':'1px',
             'cellwidth':'33.33%'
            },
            {'src':"../images/UnixL01_SHCHT/08SHLLChat.png",
             'caption':'C: Shell sends Prompt back', 
             'border': '1px solid black',
             'padding':'1px',
             'cellwidth':'33.33%'
            },
        ]
    ],
    id="fig:shell-blankline",
    caption="<center> Figure: Shell blank line behavior </center>"
)))

What happened after we pressed and release the *enter* key? The terminal sent through the Unix kernel a byte to the instance of the shell program that our terminal is virtually connected too.  Precisely, it sent the binary pattern `0b10000010` or in hex `0xa` as the left most sub-figure (A) illustrates.  The [figure](#fig:shell-blankline) show both the `\n` human readable ASCII symbol and the underlying byte value.  In other diagrams we might stop showing the byte values and only show the ASCII human readable symbol.  It is, however, important to remember that under the covers the terminal and shell are really just exchanging raw binary values that use the ASCII standard to encode the data.  If we were to use special device, such as a [logic analyzer](https://en.wikipedia.org/wiki/Logic_analyzer), to peek at the electical wires that make up the path ways that are transmitting data back and forth between the computer and the terminal we would find electical values that encode the binary for the characters being exchanged. 

The middle figure (B), cartoons the logic that the shell does in response to receiving the blank line.  Specifically, it is programmed to do "nothing", simply sending back to the terminal the prompt so that the user knows it, the shell, is ready for another command line to be sent.  If you are interested in why the terminal displayed a `\n` causing the prompt sent by the shell to be on the next line of the terminal read the fyi ECHO box below.

><img   style="display: inline; margin: 1em 1em 0px 0px;" align="left" width="50" src="../images/fyi.svg"> <p style="background-color:powderblue;"> **ECHO:**.  You might be wondering why is it that you see the characters you type at the keyboard of terminal appear on the screen of the terminal.  If pressing a key really sends the data to the Unix kernel and then to the program running, such as the shell, there is not reason we would see it.  Does the Shell send a copy back of what it receives?  No it does not.  Rather typically the UNIX kernel is configured, by default, to send back a copy of what it receives on a specific terminal connection back to the terminal itself.  This setting is called terminal ECHO mode.  So by default even before the Kernel sends data up to the destination program it sends a copy back to the terminal so that the user can see what they typed.  This includes the `\n` that the user pressed to indicate the end of the command line.  Hence, in this default mode any data sent back from the shells behavior will appear on a new-line.  Programs can ask the kernel to disable this setting to have more control of what the users sees.  As a matter of fact like many things in Unix there is a command you can issue to have the kernel adjust the setting for you terminal connect including turning off the echo behavior (`stty -echo` to turn off and `stty echo` to turn back on).  Doing so lets you observe what is really being sent back in response to your command and not what the kernel automatically generated as echo data.  Note things might get confusing quickly if you turn echo off.   But Unix is all about understanding how things work and have the power to control them.  If you want to know more try `man stty` but be warned you will find that communications on a terminal is actually quite a bit more complex than it might seem -- there is a lot of history and skeletons buried here and the kernel is *cooking* the communications quite a bit ;-)

## Command lines

UNIX command lines can get quite complex. As a matter of fact one of the hallmarks of UNIX expertise is the ability to compose long command lines that chain together many commands.  Such ability develops in time as you get familar working with the shell and learn to issue simple command lines.

The simplest command lines, other than the [blank command line](#fig:shell-blankline), is composed of a single "word".  The following are three examples of single word command lines, using the commands: `help`, `pwd` and `ls`.  



### Three simple examples

The `help` command is a "builtin" command that well provides you some help :-)

In [ ]:
bash.run("help", height='20em')

The `pwd` command is a builtin command that prints the shell's current working directory.  We will discuss what the working directory is a little [later](#wd).

In [ ]:
bash.run("pwd")

The `ls` command is a standard UNIX external command that lists the contents of a directory.  As we will find out later `ls` is a very powerful command that lets us explore the existing directories and files.

In [ ]:
bash.run("ls")

## Command line processing



% print(htmlFig([{'src': '../images/shellexecuteflowchart.png', 
%                         'border': '1px solid green'}], 
%        id="fig:flowchart", 
%        align="right", 
%        width="65%",
%        margin="1em auto 1em 1em",
%        caption="Figure: Flow chart of command line processing steps.", 
%        captionalign="left"))
<!-- produced by: 
htmlFig("[{'src': '../images/shellexecuteflowchart.png', 'border': '1px solid green'}]", 
        id="fig:flowchart", 
        align="right", 
        width="65%",
        margin="1em auto 1em 1em",
        caption="Figure: Flow chart of command line processing steps.", 
        captionalign="left")
-->
<table id="fig:flowchart" align="right" width="65%" cellpadding="0" cellspacing="0" border="0" style="border-collapse: collapse; margin: 1em auto 1em 1em">
    <tr style="padding: 0; margin: 0;"> 
        <td colspan="1" width="100%" style="padding: 0; margin: 0 0 0 0; background-color:inherit;">
            <div style="padding: 0; margin: 0 0 0 0;">
              <figure style="padding: 0; margin: 0 0 0 0; width:100%;border: 1px solid green;">
                   <img src="../images/shellexecuteflowchart.png" width="100%" style="padding: 0; margin: 0;">
                </figure>
            </div>
        </td>
    </tr>
    <caption align="bottom" style="text-align: left; padding: 0; margin: 0;" >
          <i>Figure: Flow chart of command line processing steps.</i> 
    </caption>
</table>

When the shell receives a command line it goes through a series of steps to process it.  The rules of this processing define what is called the [Shell Grammar](https://man7.org/linux/man-pages/man1/bash.1.html#SHELL_GRAMMAR).  The Shell Grammar and builtin commands form an entire programming language.  Programs written in this language are called Shell Scripts and are used extensively to automate all sorts of tasks.     Over the years one learns more and more subtle aspects and usage of the shell grammar.  While it is possible to use the shell without really knowing its underlying grammar having a working knowledge can really help in understanding why the shell behaves the way it does and its somewhat strange syntax.

The [figure](#fig:flowchat) on the right illustrates the flow of command line processing.  We will be working our way through the steps in this section.

### Interactive vs non-interactive (scripts) 

The two ways we use the shell are either interactively or non-interactive. In either case the core operation of the shell is the same -- process command lines.  The difference is in the interactive mode the commands are coming from a human though a terminal connection versus a file in the non-interactive mode.  In the interactive case after each command line is executed the shell sends the prompt to the human and waits for them to send the next command line.   In non-interactive mode the command lines are being read from a shell script file.  A shell script is a file simply containing several shell command lines. In this mode the command lines in the file are immediately executed one after the other and no prompting or waiting occurs.  

While our focus in this chapter is interactive use our understanding of how command lines are processed applies equally to writing shell scripts. 

### Overview: The six steps of command line processing

In the remainder of this chapter we will build up our working knowledge of how the shell processes command lines and its grammar.  Let's begin by breaking down command line processing into the following six steps:

1. Splitting: break up (split) the line into "blank" separated words (computer scientists also call these tokens)
2. perform expansions 
3. parse redirections
4. execute simple commands
5. optionally wait for command to complete
6. print prompt and wait for new command line if in interactive mode

### Step 1: Breaking down a "Command line" into Simple Commands and Arguments

In this step the shell splits the command line up into a set of tokens that form a "simple commands".   Several simple commands can make up a single command line. The common boundary that separates tokens are "blanks".  Blanks are `space` and `tab` characters ([blanks](https://man7.org/linux/man-pages/man1/bash.1.html#DEFINITIONS)).   In the ASCII code, space characters are encoded with a byte value of `0x20` and a tab is encoded by `0x09`.  So a sequence of one more of these two characters will cause the shell to split what is before and after the sequence into to separate tokens.  

<a id="meta_def"></a>
In addition to the blank `space` and `tab` characters the following characters `|  & ; ( ) < > newline` will also indicate the separation of tokens.  Together, these characters are known as the shell "metacharacters".  In time we will learn about how these characters affect the commands to be executed. Some will let us combine simple commands in various ways while others will effect how the command is executed.  

#### Command versus Arguments

The first token of simple command is the name of the command to execute.  The following tokens in a set will be passed to the command as arguments.     To know where the arguments of a particular simple command ends the shell looks for one of the following "control" operators.  Notice that some of the metacharacters, `| & ; ( )` and `newline` itself both separate tokens and terminate the tokens of simple command.    Using the control characters allow us to chain independent simple commands together. 

<center><em>Table Shell Control Operators</em></center>

| Control Operator | Description    |
| :--------------: | ------------- |
|  newline   |  End of line |
|      $||$        |  Or list operator   |
|      $\&\&$      |  And list operator  |
|     $;$          |  Sequential list operator |
|     $|$          |  Pipe Operator |
|     $\&$         |  Background Operator | 
|    $($ | Subshell list begin Operator |
|    $)$ | Subshell list end Operator |
|     $;;$ | Case statement end matching Operator |      
|     $;\&$ | Case statement fall through Operator |
|     $;;\&$ | Case statement continue matching Operator |

#### Comments

If a token begins with the pound character, `#`,  then it and all the remaining tokens on the command line are ignored.  As such the `#` lets us add comments to what we are doing.  This will be particularly useful when we write shell scripts.  But they be helpfull even when using the shell interactively (see [Command line history](#history)).

For example. We can ignore all tokens of the command line by placing a `#` either as a token on its own at the begining of a line,

In [ ]:
bash.run("# This line is ignored.  The first token is a # on its own.")

or by by adding to the beginning of the first token.

In [ ]:
bash.run("#This line is ignored. As the first token is #This which begins with #.")

In [ ]:
bash.run(b'# the next line ends in a comment\necho hello # rest of tokens are ignored')

Another example

In [ ]:
bash.run("pwd # print the current working directory")

#### Simple Command Line Splitting examples

Lets look at a few examples of command lines and see if we can figure out what the command and the argument are.
To help us denote spaces, tabs and newlines we will use the following notation respectively; `\ `, `\t` and `\n`.


1. Single word simple command
```bash
echo\n
```
- Breakdown: 
  - Tokens:
       1. echo
  - Command Name: echo
- Explanation:  Like our previous examples this a command line is formed of one simple one word command. In this case the ending newline both terminates the set of tokens forming our command and the line itself.   The command is the `echo` command.  `echo` is a bash builtin command that prints back its arguments followed by a newline (see `help echo`).  Given that we are not passing any additional arguments we expect echo to simple print an blank line.

In [ ]:
bash.run("echo")

2. command with one argument
```bash
echo\ hello\n
```
- Breakdown: 
  - Tokens:
       1. echo
       2. hello
  - Command Name: echo
  - Arguments:
       1. hello
- Explanation: Here our command line is composed of a single simple command which is composed of two tokens seperated by a single space.  The first token is the command and the second is the first and only argument to the command.

In [ ]:
bash.run('echo hello')

3. command with multiple arguments using multiple blanks as seperators
```bash
\t\ echo\ hello\ \ goodbye\ \t\ me\ again\ \t\n
```
- Breakdown: 
  - Tokens:
    1. echo
    2. hello
    3. goodbye
    4. me
    5. again
  - Command Name: echo
  - Arguments:
    1. hello
    2. goodbye
    3. me
    4. again
- Explanation: Here our command line is composed of a single simple command which is composed of five tokens seperated by differing combinations of spaces and tabs. To begin with we have a tab  and space preceeding the first token.  Then we have our first token `echo`  which is our command.  The next space separates `echo` from the `hello`.  Two spaces then separate `hello` from `goodbye`.  Then tghe sequence of space, tab, space separate `goodbye` from `me`.  A single space seperates `me` from `again`.  `again` is terminated as a token by the following space, tab and newline.  So it is important to note that when the shell runs echo the seperating blanks will be eliminated and only the tokens will be passed to echo as individual arguments.  

In [ ]:
bash.run(b'\t echo hello  goodbye \t me again \t')

<a id="quotes_and_splitting_sec"></a>

#### Quotes and Command Line Splitting

There are times that one might want to include spaces or tabs in an argument to a command.  For example, lets say we want to use echo to print the following as a single argument that has the spaces as part of it (not three arguments; `hello`, `...`, and `goodbye`).

```
hello    ...    goodbye
```

To do this, the shell lets us use quotes to tell it not to do splitting.  The area enclosed in either double ("") or single ('') quotes will be treated as a single token.  We will see later that the reason we might choose between using double vs single quotes has to do with the controlling the shell's expansion behaviour.  So given this to send the above to echo as a single argument we would use either of the  the following command lines.

In [ ]:
bash.run('echo "hello    ...    goodbye"')

In [ ]:
bash.run("echo 'hello    ...    goodbye'")

##### Escaping blanks

If we need finer control the shell also allows us to "escape" indivdual blank characters.  Escaped blanks will no longer be treated like a blank but instead be treated like a non-blank character.   To do this we put a back slash infront of the space or tab we want to escape. 

For example:

In [ ]:
bash.run('echo hello\ \ \ ...\ \ \ goodbye')

#### Summary

While the above can seem intimidating at first it is worth remembering that the most common kind of command lines we use on a daily bases are  composed of one simple command often with no arguments specified or at most a few arguments sparated by single spaces.  The following are examples of the kind of commands that one typically might uses.

In [ ]:
# we use a new shell for this to avoid polution of our primary bash session
bash.run('''ls
mkdir mydir
ls
cd mydir
touch myfirstfile
ls
echo "Hello.  Hello." > myfirstfile
cat myfirstfile
cp myfirstfile mysecondfile
ls 
echo "Goodbye and farewell." > mysecondfile
diff myfirstfile mysecondfile
cd''')

In all these cases the line splitting stage of command line processing is very simple and intuitive.  

<a id='history'></a>

#### Command line history

Bash keeps a history of all the commands lines we have executed.  Learning to view and navigate this history can really help speed up our daily work.    We will cover some basics to help you be productive.  You can find more details about working with your bash history here:
- Using the bash history https://www.gnu.org/software/bash/manual/bash.html#Using-History-Interactively
- searching the history https://www.gnu.org/software/bash/manual/bash.html#Searching
- keyboard short cuts for navigating the history https://www.gnu.org/software/bash/manual/bash.html#Commands-For-History


To view the entire history we can use the history command.

In [ ]:
bash.run('history')

To rerun a particular command in the history you can enter the exclamation character, `!` followed by the history number, n,  of the command line.  For example to rerun the command numbered 2 you would do the following:

In [ ]:
bash.run('!2')

A short cut for the last command is `!!`. Eg

In [ ]:
bash.run("pwd\n!!")

Bash also has some behavior that lets you avoid having a commands recorded in the history.  The defaut is that commands lines who's first character is a space will not be put into the history as well as duplicates.  (See HISTIGNORE and HISTCONTROL in the bash manual for more details on how to control and modify this behavior).  For example if we do not want our use of the history command to "polute" our history we would add a space to the front of the line.  Eg.

In [ ]:
bash.run(" history")

Finally perhaps, the most important aspect as a beginner is ability to use key sequences to: 
- go backward in the history : `previous-history`
- go forward in the history : `next-history`
- search history backwards : `reverse-search-history`
- search history forwards : `forward-search-history`

Each of these abilities are "bound" to various key sequences.  Usually previous and next will be mapped to your arrow keys.  But if you want to learn what the default "bindings" are (what key sequences will cause one of the above abilities you can use the following commands):

In [ ]:
bash.run("bind -q previous-history\nbind -q next-history\nbind -q reverse-search-history\nbind -q forward-search-history")

The above output can seem very criptic.  Here is a quick and dirty explanation.  Search the bash manual for "key bindings" for more details.

- `\C` means press the control key
- `\e` means press the escape key
- `-`  means that while pressing the preceeding key press and release the next key listed. Eg. `\C-p` means press and hold down the control key then press and release the `p` key.  
- if there is not `-` present the you press and release the keys as a sequence.  Eg. `\eOA` means press and release the escape key, then press and release the caplital o key (`O`), followed by pressing and releasing the caplital a key (`A`).

The best thing to do is to play around with these features.  Eg. try out all the prior commands and then use the history key binding to navigate the history and re-run some commands.


### Step 2: Perform Expansions

After the shell has split a command line into tokens it performs "expansion" on the tokens.  An expansion is the subtitution of some parts of a token with various other values.  In particular, it looks for specific special control characters to identify what needs expansion and what kind of expansion to do.  

Expansions add a lot of power to what you can do with command lines but they can also be a little overwhelming at firsts.    We will focus on understanding the three most common expansions.  In time as you gain greater familarity with the shell we encourage you to explore others.  

See the introduction paragraphs here https://man7.org/linux/man-pages/man1/bash.1.html#EXPANSION, and here https://www.gnu.org/software/bash/manual/bash.html#Shell-Expansions for and more detailed overview.

#### Nine Command line Expansions

Below is a brief overview of all nine expansions and our recommendations on the priority order you may want to explore them. 


##### Brace Expansion

- Priority: You can wait learn this one.  
- References: 
   - https://man7.org/linux/man-pages/man1/bash.1.html#EXPANSION (see *Brace Expansion*)
   - https://www.gnu.org/software/bash/manual/bash.html#Brace-Expansion 
- Synopsis: Provides the ability expand a "template" token in to multiple tokens. 
- Examples:

In [ ]:
bash.run('''echo poem{X,Y,Z}.text # list of values with an optional prefix and suffix
echo poem{1..4}.text # range of values with an optional prefix and suffix''')

##### Tilde Expansion

- Priority: **Simple and worth knowning**
- Synopsis: Expands the tilde (`~`) into the path of your [home directory](./files.ipynb#HOMEDIR_sec).  In time you will be navigating and moving around the directory try and it is very useful to have a short cut to the path of your home directory.
- References: https://www.gnu.org/software/bash/manual/bash.html#Tilde-Expansion, https://man7.org/linux/man-pages/man1/bash.1.html#EXPANSION (see *Tilde Expansion*)
- Examples:

In [ ]:
bash.run('''echo ~
cd ~
pwd 
cd /tmp
pwd
mkdir ~/mynewdir
ls ~
cd ~
pwd''')

##### Parameter and Variable Expansion

- Priority: **Important to know basics** - rest can wait till later
- Synopsis: Like  programming languages the shell supports the notion of a variable that is identified by a "name" and has a "value".  Shell variables are actually a special case of a what are called shell parameters. We will focus on variables. The shell provides certain special built in variables such as "RANDOM" whose value expands to a random number. As we will see later a very important built is the "PATH" variable.  There are several ways we can introduce our own variables and assign them a value.  The simplest is a command line of the form:  `name=value`.   During Parameter Expansion the `$` followed by name will trigger parameter expansion, optionally the name can be enclosed in braces `{name}`.  The token will then be expanded to the value if the named variable exists other wise the token will be simply removed.  The shell provides all kinds of fancy ways of manipulating and transforming the value of a variable during expansions, such as deleting or substituting portions, but we will leave that for later.  Finally shell variables can have attributes associated with them.   One important attribute marks a variable as exported.  We will see later that exported variables are passed to new programs as part of their initial "environment".  
- References: 
  - https://man7.org/linux/man-pages/man1/bash.1.html#PARAMETERS
  - https://man7.org/linux/man-pages/man1/bash.1.html#EXPANSION (see *Parameter Expansion*)
  - https://www.gnu.org/software/bash/manual/bash.html#Shell-Parameters
  - https://www.gnu.org/software/bash/manual/bash.html#Shell-Parameter-Expansion
- Examples:

In [ ]:
bash.run('''# set a variable who's name is x and its value to the string hello
x=hello
# $x is expands to the current value of the variable named x
echo x has the value: $x
# assign a new value to x using quotes to avoid word splitting
x="Hello and goodbye"    
# use $x again to expand x
echo x now has the value $x
# use braces to have more control 
echo x=${x}.
# here are some examples of builtins
echo $PATH
echo $RANDOM
echo $HOME
echo $SECONDS''')

##### Command Substitution

- Priority: useful but you can wait to learn about it
- Synopsis: You can specify that the output of a command should be subsituted for its name.  To trigger this behaviour, like parameter expansion you use the dollar character `$` but follow it with the command to run enclosed in brackets eg. `$(command)`.  
- References:
  - https://man7.org/linux/man-pages/man1/bash.1.html#EXPANSION (see *Command Substitution*)
  - https://www.gnu.org/software/bash/manual/bash.html#Command-Substitution
- Examples:

In [ ]:
bash.run('''# the date command prints the current time and date
date
# we can use command substitution to pass the current date string as an argument to another command
echo "The current time and date is:" $(date)
# we set a variable to the output of a command
set lastdate=$(date)''')

##### Arithmetic Expansion

- Priority: Low. Useful for doing simple math and writing loops which come in time.  
- Synopsis: Bash supports evaluation of basic arithmetic expressions as part of the expansions it does.  To trigger this expansion you must enclose the expression in double round brackets prefixed by the `$` character.  Eg. `$((expression))`.  Two particularly nice features, are the support for bitwise operators and arbitray bases.  Knowing how to use these features let's you quickly do base conversions and use bitwise operators to express values.  
- References
   - https://man7.org/linux/man-pages/man1/bash.1.html#EXPANSION (see *Arithmetic Expansion*)
    - https://man7.org/linux/man-pages/man1/bash.1.html#ARITHMETIC_EVALUATION
   - https://www.gnu.org/software/bash/manual/bash.html#Arithmetic-Expansion
   - https://www.gnu.org/software/bash/manual/bash.html#Shell-Arithmetic
- Examples:

In [ ]:
bash.run('''# does the normal interger math you expect
echo $((1+1))
x=7
echo $((x = x + 1))
echo $x
# to specify a number in a particular base use <base>#<value>
# eg. binary base 2 1100 plus base 10 3
echo $(( 2#1100 + 10#3 ))
# binary 1 left shift 12
echo $(( 2#1 << 12 ))''')


> <img style="display: inline; margin: 1em 1em 0px 0px;" align="left" width="40" src="../images/fyi.svg"> <p style="background-color:powderblue;"> While Bash lets us express values in arbitrary bases as input for arithmetic expression it uses base ten for all expanded output values. To get around this and display a value in a base other than 10 we can use the shell's built in `printf` command.  `printf` supports printing values in base 10, base 8 and base 16. Eg.  

In [ ]:
bash.run('printf "%d %o %x\\n" $((10#255)) $((10#255)) $((10#255))')

##### Process Subsitition

- Priority: low. 
- Synopsis:  This is a pretty subtle feature that lets you run commands treat them as files.   To understand what this means will require us to better understand what processes and files are.  
- References:
  - https://man7.org/linux/man-pages/man1/bash.1.html#EXPANSION  (see *Process Substitution*)
  - https://www.gnu.org/software/bash/manual/bash.html#Process-Substitution
- Examples

In [ ]:
bash.run('''# the diff command can compare two ascii files and show us the differences.
# ls lets us list the contents of a directory.  
# We can use process subtitution to treat the output of ls commands as if they where files.
# So we can use diff as follows to compare the contents of two directories
diff <(ls ~/mydir) <(ls ~/mynewdir)''')

##### Word Splitting on Expansion

- Priority: low.  
- Synopsis: After the shell has done parameter expansion, command substitution, and arithmetic expansion it will do another round of word splitting.
- References:
  - https://man7.org/linux/man-pages/man1/bash.1.html#EXPANSION  (see *Word Splitting*)
  - https://www.gnu.org/software/bash/manual/bash.html#Word-Splitting
- Examples

In [ ]:
bash.run('''x="one    two      three"
# after expanding x but before running echo the shell
# will again split the expanding output resulting in 
# this case three arguments to echo
echo $x
# we can suppress this with quotes
echo "$x"
# we can use other fancy splitting features too
y=one:two:three
OIFS=$IFS; IFS=:; echo $y; IFS=$OIFS''')

##### Filename Expansion

- Priority: **Important to know the basics** but the subtle aspects can be left for later.
- Synopsis:  A very useful feature of the shell is the ability to provide templates or patterns that will be expanded into the names of files and directories that exist and match the pattern.  The way you state a pattern and the way it is matched can get quit subtle and complex.  But the basic use is easy and can really save you time.   Ater the preceeding expansions are done and the second round of word splitting is complete the shell will scan what is left looking for a special set of characters its recognizes as pattern characters.  The most useful of these is the star `*`. The star will be substituted for any sequence of characters.  As such it is often called the wildcard character since it matches anything.  The shell will then look to see if there are files or directories that match the token subtituting the patterns as necessary.  If matching names exist then it will replace the single token with any matches found.  While this can sound complicated, in practice, the simple uses are straight forward and easy to understand from examples.  You will really want to play with this.
- References:
  - https://man7.org/linux/man-pages/man1/bash.1.html#EXPANSION  (see *Pathname Expansion* and *Pattern Matching*)
  - https://www.gnu.org/software/bash/manual/bash.html#Filename-Expansion
  - https://www.gnu.org/software/bash/manual/bash.html#Pattern-Matching
- Examples:

In [ ]:
bash.run('''# lots of commands take list of files to operate on
# eg. list, copy, word count, search
# filename expansion is very convient for flexibly generating
# list of existing directories and files.
# For example * on its own will expand to all the files that 
# exist in the current directory.
echo *
# combining star with parts of a path name can be very powerful.
# below will match all files and directories who's name start '/'
echo /*
# we could restrich this to only once that start with '/l'
echo /l*
# below will match names of file that begin with 
# '/usr/include/g' followed by anything and ends with '.h'
echo /usr/include/g*.h
# we can even use more than one '*'
echo /l*/*q*''')

##### Quote Removal
- Priority: simple and does what you expect
- Synopsis:  Finally any quotes and back-slashes not themselves within quotes or escaped will be removed.  We use generally use quotes to [control word splitting](#quotes_and_splitting_sec) and expansions (see [below](#quotes_and_expansions_sec))
This simply is the phase that removes quotes from the tokens as expected.
- References:
  - https://man7.org/linux/man-pages/man1/bash.1.html#EXPANSION  (see *Quote Removal*)
  - https://www.gnu.org/software/bash/manual/bash.html#Quote-Removal
- Examples:

In [ ]:
bash.run('''# quotes are removed from the tokens
echo "a" 'b   c' 'd' "e f"
# can nest quotes to avoid removal
echo 'my "double quote"'
echo "my 'single quote'" 
# quotes can back-slashes can be also escaped to avoid removal
echo \\'quote 1\\' \\"quote 2\\" 
echo my back slash \\\\''')

<a id="quotes_and_expansions_sec"></a>

#### Quoting and Expansions

Quotes let us control expansions in a way that is similar to how we could use quotes to [stop commandline splitting](#quotes_and_splitting_sec).  If we want to make sure that no expansions or splitting occurs over a portion of our command line we can enclose it in single quotes `'` `'`.  If we want some expansions to occur but no splitting then we can use double quotes `"` `"`. 

#### Single quotes

So when we want a squence of chacaters to be treated, as a literal, exactly as we timed them, we enclose them in single quotes.  This supresses all splitting and expansions.

In [ ]:
bash.run('''echo 'you get {1..4} /* exactly !#$@$ $what $((1+1)) you type'
''')

#### Double quotes

Doubles quotes provide us with something in between no quotes and single quotes.  They let us ensure that the enclosed text will be treated as a single token however cetain expansions are still processed prior to the command being executed.  All expansions that use the `$` as a prefix will still occur. 


### Step 3: Parse Redirections

To fully understand what it means to parse redirections we must have a more complete understanding of how commands are run and the UNIX I/O model -- how a program can conduct input and output.    This understanding will take us a little time as it involves concepts such as Processes, Streams and Files and how they inter-relate.  At this point we take a simple view that we will build upon in time.  

#### Processes

As we go our understanding of what a process is will become more and more advanced.  For the moment our simple working definition will be a running program other than the kernel.   The shell we interact with is a process and the commands that we run from the shell run within it or as new processes.   We will look at this distinction in more detail when we discuss internal built-in versus external commands.   Regardless the commands we issue run as a process.

The Parse Redirections step is is all about controlling where the input and out for a process, comes from or goes to.  

#### Streams: Standard Input, Output and Error

UNIX provides three default "streams" that a process can use for its input and output.  The default streams are called: Standard Input, Standard Output and Standard Error.  A process can ignore these if it wants but most commands by default use these streams.  A stream is a connection to a source or destination for reading or writing bytes.  A process read bytes from a stream via `read` calls to the kernel. Such a stream is an input stream.  Similarly a stream that a process can write bytes to, via `write`  kernel calls, is an output stream.   As implied by the name a stream is like a "flow" of bytes.  Reading bytes from an input stream successively gets bytes from the source it is connected to.    Writing bytes to a stream successively sends more and more bytes to the connected destination.  


#### ASCII streams

What the bytes "mean" or what will be done with them is up to the process. Most of the daily UNIX commands we use assume ASCII line oriented data.  This means that most commands expect that the bytes they read from their standard input will be bytes that encode ASCII lines of data.  Similarly we can expect most commands to write ASCII lines of data to their standard output and error streams.  

##### Standard Error

The UNIX designers knew that it is very common to have commands produce "normal" output and "error" output.  To support this separation they decided to make it standard that processes get both, a standard output and error stream.  

By convention when UNIX commands are being developed by a programmer they write them to send their "normal" output to the standard out stream.    When the programmer of a command wants to send error messages or warning messages they use the standard error stream.  This allows some one using a command to easily direct normal output to one destination and error output to another.   Eg. they might decide they want to ignore and hide all warning messages by redirecting the error stream to a different destination.   While for the most part we don't normally use this extra flexibility there are times when it comes in is very handy.  


#### Streams and Files

But where exactly do the bytes for a stream come from or go to?
A common phrase that you may hear is that in UNIX everything is a file.  From this perspective streams are connections to files that allow a command to read or write the data of the file.   

The designers of UNIX decided that they would try and represent all locations that might serve as a source or destination of data as files.  This includes even devices such as terminals.  To this end the UNIX kernel represents the majority of the devices of a computer as files that a stream can be connected too.   Reading from an input stream connected to a device "file" gets data from that device.  Writing to an output stream connected to a device file sends data to the device.

#### Opening streams and File Descriptors

To connect a stream a program makes an "open" kernel call.  As part of the open call the program passes the path name of the file to connect the stream to and also specifies if the connection is for input or output (or both).  If successful the kernel will establish the connection.  

For every process, running program, the kernel maintains a table of open streams. This table is called a file descriptor table.  Each entry in the table is either empty or not empty.  If empty the entry does not hold a description of an open stream.  If it is not empty then it does contain the information that describes an open stream.  The kernel can then use this descriptor information to operate on the stream, eg. read or write bytes to it.

When an open call is made the kernel scans the table for next empty location and then stores the descriptor information about the newly open stream into that location.  The index of this newly populated table entry is returned back to the process.  

Historically these indexes are themselves called **File Descriptor** numbers.  All the kernel operations on a stream, such as read and write, require a file descriptor so that the kernel knows what stream to operate on.  

The first three  file descriptors; 0, 1, and 2 are the where connections for standard input, output and error are expected to be. 

> <img style="display: inline; margin: 1em 1em 0px 0px;" align="left" width="40" src="../images/fyi.svg"> <p style="background-color:powderblue;">  The UNIX kernel also provides other operations that lets a process control its file descriptors.  This include the ability to close and mark one as empty via the `close` kernel call.  It also lets a process control its table of file descriptors via kernel calls such as `dup` and `dup2` that duplicated an table entry.  


#### Terminals and Streams

As we read an ASCII terminal provides a natural way for humans to both receive and send ASCII line oriented data.  When we connect to a UNIX system via a new terminal the kernel starts a shell for us and connects its standard streams to the particular device file that represents our terminal.   When the shell writes bytes such as the prompt to its standard output we see it in the terminal it is "connected" too.  Similarly when we type characters into the terminal the shell will receive them when it reads from its standard input.  


#### Defaults

When the shell runs a command it establishes what the commands standard input, output and error streams will be connected to.  The default is to connect them to the same streams that the shell is connected to.  In most cases this will be the terminal that the shell is interacting with.    This is the reason the input and output of a command, by default come from and go to the terminal, we are working within.  


#### Overriding the defaults

Once steps 1 and 2 of command line processing have completed the shell has identified one or more commands along with arguments that we want executed.   In addition to commands and their arguments the shell gives us the ability to control where the input and output streams for a program might come from and go to.
    
As stated above if we don't use any of the special redirection support then the defaults will be used.   If however we use the redirection syntax then we can have the shell control what file descriptors a command will have open and where they will be connected to before they run.  While there are many advanced uses of the redirection syntax basic usage is straight forward.  
    
When discussing command line splitting we introduced that `<` and `>` are <a href="#meta_def">metacharacters</a>.  Their primary use is to specify redirections.  The greater than character,  `>`,  is used to control output stream redirection. Conversely, the less than, `<` character is used to control input stream redirection.
  
The following examples introduce the basics. See the references for details of the syntax and more advanced uses.

References:
  - https://man7.org/linux/man-pages/man1/bash.1.html#REDIRECTION
  - https://www.gnu.org/software/bash/manual/bash.html#Redirections
  
  
#### Redirection Examples
    
In theses examples we will use  the `date`, `ls`, `cat`,  and `file` commands.  
  
- The `file` command, tells the "type" of the file that we specify as an argument.  We will use it to tell when we a file is a device file versus a regular file. 
- The `cat` command, by default, if no arguments are specified, reads bytes from the standard input stream and writes them to the standard output stream.  
- The `ls` command, as we have seen, lists the contents of the specified directory to standard output.  By default, if no arguments are given then `ls` will list the contents of the current working directory.  
- Finally the `date` command, if no arguments are given prints the current date and time to the standard output stream.

##### Standard output and redirection

Let's start with the basics.  `date`

In [ ]:
bash.run('''date''')

A command line of `date` command alone displays its output to us.  But why? Further how can we change this?

The `bash` process that launches the date command as a new process provides this new process with a copy of its open streams.  An interactive bash process that is connected to a terminal will have its streams 
- standard input stream (fd 0), 
- standard output (fd 1), and 
- standard error (fd 2) 

All connected to a device file that the kernel provides to represent the terminal the `bash` process was started from.

As it turns out we can use a  command line to determine what files a process's standard streams are connected too on Linux. Specifically, the following will let us determine what files the standard streams, of the current bash we are interacting with, are connected too:

```bash
ls -l /proc/$$/fd/{0..2}
```

We will deffer understanding exactly how this command line works till later when we discuss process identifiers.

In [ ]:
bash.run('''ls -l /proc/$$/fd/{0..2}''')

In [ ]:
bash_pid=bash.getPid()
bash_stdout=os.path.realpath("/proc/" + bash_pid.__str__() + "/fd/1")
bash.run("file " + bash_stdout)

In [ ]:
display(Markdown("What we see is that all three file descriptors (0,1,2) are pointing to the same file (" + bash_stdout + ") .  Using the `file` command we can find out if this file is a special device file or a regular file.")  )

Indeed we see that it is a "character special" file.  Which is the type of special file that terminals are in UNIX.    With this in mind let's explore the standard output redirection.

Using the redirection syntax the following is the equivalent to what is happening by default. 

In [ ]:
bash.run("date > " + bash_stdout)

Namely the process started to run date has its standard output connected to the same file that the shell process that launched it has as its standard output destination. 

Now using what we know we can ask the shell to set the destination for standard output to a different file.  Doing so ensures that any output the command sends to its standard output stream (fd 1) will automatically be sent to the new destination. Eg.


In [ ]:
bash.run('''ls
date > mydate
ls''')

What we see that by providing our own path name for the destination file the shell:
1. created the new file
2. ran the date command in a new process with the new file connected to the process's standard output

As such we don't see any output from `date > mydate`.  Instead we see that the new file exists and we assume its contents contains the output from running `date`.  To confirm this we can use the `cat` command.  Passing file names as arguments to `cat` causes it to open the specified files and copy their contents to standard output.  

In [ ]:
bash.run("cat mydate")

In [ ]:
display(Markdown("Given what we know, the above is really " + 
                 "copying the contents of `myfile` to the default " + 
                 "file connected to the standard output of the process running `cat`. " + 
                 "Which of course is the file the shell connects to the process standard output, " + 
                 "defaulting to the shell's standard out destination." +
                 "  Which we saw in our case to be `" + bash_stdout + "`."))

Which means that the above is the equivalent to:

In [ ]:
bash.run("cat mydate > " + bash_stdout)

As you can see, given that "everything is a file", the `cat` command, which simply copies bytes to its standard output becomes very powerful given redirection.  

Using redirection how could we copy the contents of `mydate` to a file called `mydate.old`?

In [ ]:
Answer('''
> `cat mydate > mydate.old`
''')

###### Appending to a file

Before we go on to look at standard input redirection.  Let's look at what happens if we redirect to an existing file.  When we use the `>` syntax  the shell will:
1. As we saw, if the file does not exist it will create a new file and direct all output of the specified command into that file.
2. However, if the file does exist it will open the file and truncate it's contents before setting the file up as the standard output.   In other words, redirecting to an existing normal file will replace the file's entire contents with the new output. 

For example:

In [ ]:
bash.run('''cat mydate
date > mydate
cat mydate
sleep 10 # use the sleep command to wait 10 seconds
date > mydate
cat mydate''')

However, there are times that we might want to add to the end of an existing file rather than truncating it and replacing its contents.  To do this we use two greater than symbols instead of one.  Eg. `date >> mydate`.  In this case the shell will not truncate the file instead it will open the file for appending ensuring that the output of the new command is added starting at the current end of the file.  


In [ ]:
bash.run('''cat mydate
date >> mydate
cat mydate
sleep 10 # use the sleep command to wait 10 seconds
date >> mydate
cat mydate''')

```{note}
>**nocolobber:** As we saw the default behavior when we redirect output to an existing file is to truncate and replace its contents.  In other words we silently "clobber" the file.  You can override this default behavior so that an error is reported if a redirection would "clobber" the contents of an existing file.
See https://www.gnu.org/software/bash/manual/bash.html#Redirecting-Output and https://www.gnu.org/software/bash/manual/bash.html#The-Set-Builtin (specifically the -C option) 
```

##### Standard Input redirection 

Similar to Standard output redirection we can ask the shell to override the file that is connected to a commands standard input stream (fd 0).   The symbol we use for this is the less than character `<`.  

For these examples we need to now look more carefully at how UNIX programs deal with input.    When we provide no arguments to `cat` it will read bytes from standard input and send its contents to standard output.  Given what we know then we can copy bytes from our terminal connection to a file by simply running.  

In [ ]:
cat = BashSession()
cat.rawWriteAllOutput('''cat
''', sendEOF=False)

Now any line we send from the terminal, connected to the standard input of `cat`, will be copied to it's standard output, which again is the terminal.  As such we see a copy of the line back on the terminal.

In [ ]:
cat.rawWriteAllOutput('''hello
''', sendEOF=False)

Now every line we type will continue to be sent to `cat` and hence get sent back to the terminal

In [ ]:
cat.rawWriteAllOutput('''goodbye
''', sendEOF=False)

To end the `cat` process, and return back to interacting with the shell, we must ask the kernel to simulate an "end of file".  To do this we can press the `control` key and `d` at the terminal ('ctrl-d').  This tells the kernel to signal to `cat` that its standard input stream  is at it's end.   Which in turn caused `cat` to exit and for us to get back to interacting with `bash`. 

In [ ]:
cat.rawWriteAllOutput('')

In [ ]:
del cat

So now we can use `cat` and redirection to send data from our terminal to a file.

In [ ]:
cat = BashSession()
cat.run("ls")
cat.rawWriteAllOutput('''cat > myfile
hello
goodbye
''')

Now we can use `cat` and redirection to send the data from the file back to the terminal.

In [ ]:
cat.run('''ls
cat < myfile''')

Add of course use `cat` append to the file

In [ ]:
bash.rawWrite('''cat >> myfile
some more text
to be or not to be...
''')

In [ ]:
cat.run('''cat < myfile''')

And finally we can combine input and output redirection to copy regular files.

In [ ]:
bash.run('''cat < myfile > anotherfile''')

In [ ]:
bash.run('''ls
cat < anotherfile''')

We can specify basic redirection in any order we like.

In [ ]:
bash.run('''cat >> anotherfile < myfile
cat < anotherfile''')

In [ ]:
del cat

##### Pipes

The UNIX kernel provides another very powerful model for using redirection called pipes. The basic idea is to allow the output from one running program to be connected to the input to another running program.  This is accomplished by combining output and input redirection of two processes through a special kernel provided file called a pipe.  

The shell offers special syntax for exploiting pipes to start and connect several programs to form a chain of processes called a pipeline.   Pipelines are a hallmark of the UNIX command line environment and we discuss them more detail in the [Pipelines](#pipeline:sec) section. 

##### Standard Error redirection

Shells also typically permit you to redirect standard error independently from standard output.  In `bash` this can be accomplished by the more general syntax of `n>` where `n` is a file descriptor number.   Using this syntax we can redirect standard error with `2>`

For example Lets assume that the file `foo` does not exist.  When we run `cat foo mydate`, `cat` will send an error message to standard error saying that the file `foo` does not exist and then go on to open `mydate` and send its contents to standard output


In [ ]:
bash.run("cat foo mydate")


To redirect the error output to an separate file we can use the following:

In [ ]:
bash.run('''cat foo mydate 2> errors
ls
cat errors''')

Similarly we can append using `n>>`.

In [ ]:
bash.run('''cat errors
cat mydate bar mydate goo 2>> errors
cat errors''')

> <img style="display: inline; margin: 1em 1em 0px 0px;" align="left" width="40" src="../images/fyi.svg"> <p style="background-color:powderblue;"> In addition to the devices files that represent hardware such as terminals, printers, etc., the kernel also provides some really useful special files that we can use.  Here are a few: 1)`/dev/null` - anything written to this file is thrown away.  This can be really useful when you want to silence the output of a command. 2) `/dev/zero` - This file is an infinite stream of zero valued bytes. 3)`/dev/random` - This file is an infinite stream of random valued bytes. In time you will find that combining such files with the standard UNIX commands and redirection lets you do a lot.

### Step 4: Execute the command

Once we reach step 4 of command line processing the shell will have:
1. identified the names of commands to be executed
2. a set of arguments, if any, to pass to the commands
3. how to directed the input and output streams of the commands

As an example consider the following command line:

```bash
ls /etc > ${HOME}/etc.txt 
```

At the start of step 4 the above will have been broken down into the execution of a single command:

- **command name:** ls
- **arguments:**
    1. /etc
- **redirections:**
    - **standard out:** /home/jovyan/etc.txt
    
        Where `${HOME}` triggered variable expansion.  The value of the `HOME` shell variable is the path name of the users home directory.  In our case it happens to be `/home/jovyan`.  This could be a different value depending on the UNIX system you are on and your user name.

The goal now is to  "execute" the command passing the arguments with the appropriate redirection setup. 

```{important}
The way this step works is critical to the flexibility and power the shell provides. It is why as a programmer you can easily extend and customize the UNIX command line environment with the programs and scripts you write.
```

Let's now recall the flow chart of command line processing so that we can focus on the sub-steps of command execution.

In [ ]:
display(HTML(htmlFig(
    [[
        {'src':"../images/shellexecuteflowchart.png",
         'cellwidth': '80%'},
     ]],
    caption="Figure: Flow chart of command line processing steps."
)))

In the case of our example command line the following outlines how execution would proceed in terms of the flowchart of sub-steps.
- *DECISION 1:*  The command name `ls` does not have a `/` in it so we would proceed from decision one to decision two.  
- *DECISIONS 2 & 3:* As it happens `ls` is not a user shell `function` nor a `builtin` command. So we will proceed to decision four.  
- *DECISION 4:* In our case `ls` matches the name of file the `/usr/bin/ls` as `/usr/bin` is listed in our PATH variable.  As such the command `ls` triggers step B: "execute as external command" 
- *STEP B:* The shell requests the kernel to start the executable `/usr/bin/ls` as a new process.  
    - With `/etc` passed as the first argument to the new process and 
    - with the standard output of this process being directed to a file `/home/jovyan/etc.txt`.
  
  Assuming all goes well we would end up at decision 7.   
- *DECISION 7:* By default all commands are started synchronously so we would proceed to the final read step.  
- The shell would wait for the newly created `ls` process to exit, gathering its exit status.

At this point we don't know how to exactly how to verify the above nor what each decision and step means.  The following discusses the execution sub-steps in more detail and how they interact.  

#### Builtin vs Externals

The two core types of command execution are:
1. builtin  (steps A)
2. external (step B) 

##### Builtins
Builtin commands are those that are directly part of the shell program.  In other words the shell has a name of an internal command (builtin) that matches the command name specified. 

Examples of these include: `help`, `echo`, `cd`, `print`.  To see what builtin commands are available you can use the `help` command.  

In [ ]:
bash.run("help", height='10em')

As stated in the output of the help command you can use `help <cmd>` to get more information about a particular command.  

In [ ]:
bash.run("help echo", height='10em')

```{important} 
Note the help command also points you to other great resources (the `info` and `man` commands)
```

##### External "Shell Commands"

Perhaps one of the coolest things about the shell, and more generally UNIX, is the way everything is broken down into small building blocks.  Rather than building the majority of the UNIX commands into the shell the designers choose to provide an easy way of making additional external programs automatically found and launched.  As matter of fact, philosophically, the original idea was to keep the number of builtins as small as possible. 

In this way the job of the shell stays focused on the syntax for interacting with a human user and the launching of additional commands as processes.  The external programs that one adds to the system can each focus on adding a single new ability.  Eg. A program that lists the contents of a directory to its standard output (`ls`), a program that copies its standard input stream to its standard output stream (`cat`), a program that lists all the current processes (`ps`).  
This "separation of concerns" not only allows for a great deal of flexibility in adding commands, it also means that one can have several different version of shell programs.  Where each shell version provides its own take on the syntax but they all can reuse the existing commands.  


This separation of concerns approach, empowers and encourages programmers:
- A programmer can easily grow the system by added a new "command"
- A programmer can focus each new command she writes on a single task.  
- A programmer can easily override the default commands by simply installing their own programs which are found before the standard one

Over the years the flexibility of this approach and the popularity of UNIX has lead to the development of a very large set of programs that one can add to the system and treat as commands.  Although they are often referred to as *"Shell Commands"* they are not really builtin to an given shell.  

Today, when people talk about knowing UNIX commands they, generally are referring to a set of external programs that have evolved into a standard.   The following Wikipedia page documents such a list.

In [ ]:
display(IFrame("https://en.wikipedia.org/wiki/List_of_Unix_commands", height=600, width="100%"))

##### Locating  External commands

There are two ways to have the shell run an external file as a program:
1) by using the pathname of a file as the command
2) by using the name of a file that is in on of the directories listed in value of the shell variable called PATH.

###### Commands as pathnames

One way to execute a external command,  indicated by DECISION 1 on the flow chart, is to use the "path" name of the file we want to execute as a command.   Recall that [pathname](../unix/files.ipynb#pathname) of a file in UNIX is a name that uses `/`'s to indicate its name withing the tree of existing directories and files.  DECISION 1 is really saying that if the command name of the command line is something that "looks" like a path name then assume it is.  In particular ask the kernel to start a new process from the file.


```{admonition} Absolute and Relative Paths
In the prior chapter, [Organizing and Managing Information](../unix/files.ipynb), we discussed how a file is named as a pathname starting at the Root (`/`) of the directory tree.   Such paths whose names begin with `/` are called absolute paths.  
To ease the burden of naming files UNIX also provides what are called relative paths.  Rather that starting a path name all the way from the root of the directory tree,  a path can be named relative.  As is discusses in the [Environment](#cwd) section each process has associated with it a "current working directory".  We can think of this as a pointer to a specific directory in the directory tree.  When a process asks the kernel to open or lookup a path that does not start with a leading `/` the kernel will assume that the path is relative to the processes current working directory. 

When we start a new terminal session the shell started has its current working directory initially set to our users HOME directory.  Eg. For the jovyan user in our UNIX environment it would be /home/jovyan.  The `pwd` command can be used to print what the shell's current working directory is and the `cd` command can used to change it to another directory. 

So if we use, `mydir/myfile`, as a shell command.  The Shell will asked the kernel to locate as an external command.  Given the lack of a leading `/` the kernel will assume that we are looking for a file, `myfile` in a directory called `mydir` which is under the the shell process's current working directory.  If the working directory is `/home/jovyan` then the full path of the file we are trying to locate would be `/home/jovyan/mydir/myfile`.

Notice, however, if our current working directory was `/home/jovyan/mydir` and we wanted to have the shell execute the file `myfile` within it as an external, by pathname, it would not work to simply type `myfile` as the command.  DECISION 1 explicitly states that to treat the command name as a pathname for an external program the command name must contain a `/`.   As it turns out every directory in UNIX contains two hidden directories `.` and `..`.  Where `.` is a short cut for the name of the directory and `..` is a short cut for the name of the parent of the directory.  Using this feature we can construct a name of a file within the current working directory that contains a slash by using `./` as a prefix.  Eg. to trigger the explicit request to execute `myfile` in the current directory as an external command we can use `./myfile`. 

Details of how path names are decoded can be found in the "path_resolution" man page (`man path_resolution`).  
```

###### Commands as files within the PATH list

Using pathnames lets us ask the shell to try and execute an file as a process.  However, this does not really make external programs feel like builtin commands.  

Notice that there is another path way in the flow chart for reaching STEP B external command execution.  Specifically if the command name is not a user defined function (see [Functions](#functions)),  or a shell builtin command, then the shell will look to see if the command is the name of a file within a set of directories.  The set of directories is the current value of the shell's PATH variable.  Specifically, the PATH variable is treated as a colon separated list of directories to be searched for files that will be launched as external commands.  The list is search from left to right.   If the name of the command matches the name of a file within the directory being searched then the file will be treated as an external command and the shell will attempt to execute it.  Note the first match will terminate the search.  So even if directories later in the path also have a file that matches the command name they will not be considered.

Notice three critical points to this design:
1. Files within the set of directories do not require use of path names to be executed.  This makes them feel more like built in commands.  Simply adding files to one of the directories listed by the PATH variable is enough to make the program become a "command".
2. The PATH variable is just an ordinary shell variable that can be updated any time one likes.   You can set it to any values you like.  Thus you can easily add directories of files to it and or remove directories from it. Further remember that every terminal connection has an independent shell process connected to it, each with their own version of the PATH variable.  As such you can override the PATH in one shell process while leaving it untouched in another. 
3. By controlling the order of directories you can over ride which version of a file should be launched as the command.  In this way you can provide alternative "personalities" to the commands by have your own versions in directories that appear before the directories with the standard versions. 

Lets look explore this in a little more detail by examining how the `cat` command gets located.  First lets see if `cat` is a built in by asking for `help` about it.

In [ ]:
bash.run("help cat")

Sure enough `cat` does not seem to be a built in.  Let's look at the PATH variable to see what is going on.

In [ ]:
bash.run("echo $PATH")

We see that PATH variable lists several directories (each directory is an independent element of the list where elements are separated by a `:`).  While we could use ls to search each element the shell builtin command `type` has a convenient option `-a` that displays all the locations containing an executable (see `help type` for more details).

In [ ]:
bash.run('type -a cat')

We see that there are two directories listed in the current PATH varaibles value that contain a file named `cat`.  Given that `/usr/bin` appears first in the path, from left to right, means it is this file that will currently be executed when we use `cat` as a command name.  We can use the `-p` option to type to confirm this as it will print the one that currently will be executed.

In [ ]:
bash.run('type -p cat')

Remember that all the files in all the directories of PATH variable can be treated as commands.  The following lists all of them.  

In [ ]:
bash.run("ls ${PATH//:/ }", height="10em")

We can use the following command to get an approximate count.

In [ ]:
bash.run('ls -1 ${PATH//:/ } | wc -l')

That's quite a lot.  One of the joy's of UNIX is learning to poke around and explore.  We encourage you to explore the directories and files that you find.

Let's now exploit the flexibility of the shell to provide our own version of `cat`.  For this example we need to use a few command and ideas that we have not yet covered.  We will provide brief explanations and defer more detailed coverage till later.

Lets first begin by creating a directory, called `mybin`, within our home directory, to place our new version of `cat`.  To do this we will use the `mkdir` command

In [ ]:
bash.run('''mkdir ~/mybin
ls ~''')

Our new `cat` command will be a very simple shell script.  Which is simply a file that contains shell commands.  To do this we will use `echo` and redirection to create the file and put the contents into it.  To be able to have the kernel be able to execute the file as a shell script we will do to things.  First we will make the file line be the path of the shell we want the kernel to execute in a new process to "interpret" the rest of the contents as its input.  The next line of the file will be and the shell command we want executed.  The second thing we must do is mark the meta-data of file as executable so that the kernel will identify it as a file to execute.  To do this we will use the `chmod +x` command.


In [ ]:
bash.run("echo '#!/bin/bash' > ~/mybin/cat")

In [ ]:
bash.run('''echo 'echo "One cat just leads to another. – Ernest Hemingway"' >> ~/mybin/cat ''')

In [ ]:
bash.run("/usr/bin/cat ~/mybin/cat")

In [ ]:
bash.run("chmod +x ~/mybin/cat")

Using `ls -l` we can inspect the meta-data of the our new file and confirm that it is now marked as executable.  Notice the `x`'s in the data to the left.  We will discuss file permission in more detail later.

In [ ]:
bash.run("ls -l ~/mybin/cat")

Ok lets start by trying to run out new version of cat useing its path name.


In [ ]:
bash.run("mybin/cat")

Ok.  Now lets add our `mybin` directory  to the path and see what happens.

In [ ]:
bash.run("PATH=${PATH}:/home/jovyan/mybin")

Lets check to see what the path looks like now

In [ ]:
bash.run("echo $PATH")

Lets now try using `cat` as our command

In [ ]:
bash.rawWrite('''cat
''')

But of course this does not work as we put the directory at the end.  Lets see what `type` has to say.

In [ ]:
bash.run('''type -a cat
type -p cat''')

Let's be more careful and put our directory at be beginning

In [ ]:
bash.run('''PATH=/home/jovyan/mybin:$PATH''')

In [ ]:
bash.run('cat')

Cool!  But how do can we use the system `cat`? Well we can use the full path eg. `/usr/bin/cat` or we can put PATH back to the way it was.

In [ ]:
bash.run("PATH=/opt/app-root/src/juphub-utils:/opt/conda/bin:/opt/conda/condabin:/opt/app-root/src/juphub-utils:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin")

Now using the above as an example try and add an external command that is a simple shell script called `hello` that when you use `hello` as a command the the string "Hello there!" is printed to the standard output stream.

##### Executing External commands (Creating Processes)

Now that we have see how files can be located and treated by the shell as external commands lets look a little close at how they are executed as new processes.



### Step 5: Optionally wait for command to complete

### Step 6: Print prompt if interactive

## Commands

There are four categories of shell commands: 1) Simple Commands, 2) Pipelines, 3) List of Commands, 4) Compound Commands.  Simple Commands are the bread and butter of how we use the shell and what we focus on in this chapter.  Compound commands allow one to group simple command together into a single unit.  Compound commands add

### Simple Commands
As one might expect, the heart of shell command lines are commands. A command is identified by a name formed from ASCII characters.  In our simple examples above these were, `help`, `pwd` and `ls`.   There are two core types of commands:
1. builtin commands
2. external commands

To truly understand the Unix development environment and the philosophy behind it one needs to understand the distinction between shell built-in commands and external commands/programs.

### Variable Assignments

As part of the processing Simple Command the shell support the ability to set "variables".  (

#### Built-in commands 

As we saw t

### Pipelines

### List of Commands

### Compound Commands

#### Loops

#### Conditionals

#### Groups

## User defined shell Functions


## The Environment

### Working directory

### Exported Shell Variables

### Open File Descriptors


# Summary 

Now that we have a sense for the basic model of interacting with the shell, Unix's ASCII oriented human interface, we can go on and learn about some of the actual commands, both internal and external.  Along with the commands are common ways to use them to get things/tasks done.

## Exercises

### Given the following command line determine how it will be split into tokens and which token is the command and which are the arguments.
Again we use `\ `, `\t` and `\n` to indicate space, tab and newline characters respectively.
```
\ foo\t\t\t\bar \blah\ goo\n 
```


In [ ]:
Answer('''
> There are four tokens: `foo`, `bar`, `blah` and `goo`.  The command token is `foo`. The arguments to `foo` will be; 1. `bar`, 2. `blah` and 3. `goo`
''')